## Gdsfactory v8 Upgrade

Gdsfactory v8, based on KFactory, offers enhanced routing functions and additional features from KLayout, including DRC, dummy fill, and connectivity checks.

For those still using gdsfactory v7, it is hosted in the gdsfactory7 repository: [https://github.com/gdsfactory/gdsfactory7](https://github.com/gdsfactory/gdsfactory7), along with the [documentation](https://gdsfactory.github.io/gdsfactory7/).

### Benefits of Migrating:

- **Integrated Data**: Ports, information, and settings are stored within the GDS file, eliminating the need for separate files.
- **Improved Booleans**: Booleans are more robust with integer-based polygons, removing slivers of less than 1nm.
- **Enhanced Features**: More robust booleans, DRC, LVS, and connectivity checks.
- **Active Maintenance**: More actively maintained with frequent updates.
- **Advanced Routing Algorithms**: Better routing algorithms for efficient design.
- **Grid Alignment**: Ports and polygons snap to grid by default, reducing the likelihood of 1nm gaps.

### Drawbacks of Migrating:

- **Potential Errors**: As with any code changes, undesired errors may be introduced. It is recommended to have regression tests for all your components.
- **Non-Manhattan Placement**: Slightly more challenging.
- **Incomplete Functionality**: Some features, such as `route_path_length_match`, are not yet implemented.

### Major Differences:

- **Coordinate System**: `xmin`, `xmax`, `ymin`, and `ymax` use integers in Database Units (1nm by default). To set/get them in um, use `d` (decimal) e.g., `Instance.dxmin` which is in float.
- **LayerMap**: Now an Enum of integers.
- **Routing Functions**: New functions do not require starting ports to be in the same orientation and monitor for self-intersections. `get_route` is now `route_single`, and `get_bundle` is now `route_bundle`.
- **Grid Snapping**: All polygon points snap to grid, mitigating 1nm gaps.

### Minor Differences:

- Replace `from gdsfactory.cell import cell` with `from gdsfactory import cell`.

### How to Migrate:

We provide a migration script to assist with migrating your code. Ensure to verify any automatic migrations, and ideally, write regression tests for your code before migrating to ensure accuracy.

```bash
gf migrate -i --migration 7to8 <input_folder/file> 
# or 
gf migrate --inplace --migration 7to8 <input_folder/file>


## Use dxmin, dcenter, dmove, dsize_info

This is the biggest difference. By default `xmin`, `ymin`, `move` and `size_info` will change from um (Decimals) to Database units in the next major gdsfactory release.

In [ ]:
import gdsfactory as gf

c = gf.Component()
ref = c << gf.components.bend_euler(radius=5)
c

In [ ]:
print(ref.dxmax)

As you can see you can still use `ref.xmax` but you will get a deprecation warning.

In [ ]:
print(ref.xmax)

In [ ]:
print(ref.dsize_info.width)

## Use instance.drotate 

For setting arbitrary rotation angles you can use `Instance.drotate(30)` to rotate 30deg as an example.

In [ ]:
c = gf.Component()
ref = c << gf.components.bend_euler(radius=5)
ref.drotate(45)
c

## LayerMap 

In v7 or below, a LayerMap needs to be called

```python

from gdsfactory.technology import LayerMap

class LayerMapFab(LayerMap):
    WG = (1, 0)

LAYER = LayerMapFab()
```

However in v8 it has a different type and does not need to be called 

```python

from gdsfactory.technology import LayerMap

class LayerMapFab(LayerMap):
    WG = (1, 0)

LAYER = LayerMapFab
```

See below:

In [ ]:
from gdsfactory.technology import LayerMap


class LayerMapFab(LayerMap):
    WG = (1, 0)


LAYER = LayerMapFab
type(LAYER)

In [ ]:
LAYER.WG

In [ ]:
tuple(LAYER.WG)

In [ ]:
str(LAYER.WG)

## Routing functions

Routing functions do no longer return the route Instances but they place the instances in a Component, so you have to pass a Component.

In [ ]:
c = gf.Component()
w = gf.components.straight(cross_section="rib")
top = c << w
bot = c << w
bot.dmove((0, -2))

p0 = top.ports["o2"]
p1 = bot.ports["o2"]

r = gf.routing.route_single(
    c,
    p0,
    p1,
    cross_section="rib",
)
c

🚀 The new routing functions allow the starting ports `ports1` to have different orientations.

The end ports still require to have the same orientation.

In [ ]:
c = gf.Component()
top = c << gf.components.nxn(north=8, south=0, east=0, west=0)
bot = c << gf.components.nxn(north=2, south=2, east=2, west=2, xsize=10, ysize=10)
top.dmovey(100)

routes = gf.routing.route_bundle(
    c,
    ports1=bot.ports,
    ports2=top.ports,
    radius=5,
    sort_ports=True,
)
c

In [ ]:
c = gf.Component()
columns = 2
ptop = c << gf.components.pad_array(columns=columns, port_orientation=270)
pbot = c << gf.components.pad_array(port_orientation=270, columns=columns)

ptop.dmovex(300)
ptop.dmovey(300)
routes = gf.routing.route_bundle_electrical(
    c,
    reversed(pbot.ports),
    ptop.ports,
    start_straight_length=100,
    separation=20,
    bboxes=[ptop.bbox(), pbot.bbox()],  # can avoid obstacles
)

c

In [ ]:
c = gf.Component()
columns = 2
ptop = c << gf.components.pad_array(columns=columns, port_orientation=270)
pbot = c << gf.components.pad_array(port_orientation=270, columns=columns)
ptop.dmovex(300)
ptop.dmovey(300)

obstacle = c << gf.c.rectangle(size=(300, 100), layer="M3")
obstacle.dymin = pbot.dymax - 10
obstacle.dxmin = pbot.dxmax - 10


routes = gf.routing.route_bundle_electrical(
    c,
    pbot.ports,
    ptop.ports,
    start_straight_length=100,
    separation=20,
    bboxes=[
        obstacle.bbox(),
        pbot.bbox(),
        ptop.bbox(),
    ],  # will ba able avoid obstacles soon
    sort_ports=True,
)

c